<a href="https://colab.research.google.com/github/hobbit0701/hobbit/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.3 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
from flask import Flask
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask,request,abort

from linebot.v3 import(
    WebhookHandler
    )
from linebot.v3.exceptions import(InvalidSignatureError)
from linebot.v3.messaging import(
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage,ImageMessage,VideoMessage,LocationMessage
)

from linebot.v3.webhooks import(MessageEvent,TextMessageContent)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route('/callback',methods=['POST'])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'
import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent,message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action =='sticker':
      reply=StickerMessage(package_id='446',sticker_id='1989')

    elif action =='image':
      reply = ImageMessage(originalContentUrl='https://thumbor.4gamers.com.tw/ScSRflOzjgQwfoiNq4AhSDvegwY=/adaptive-fit-in/1200x1200/filters:no_upscale():extract_cover():format(jpeg):quality(85)/https%3A%2F%2Fugc-media.4gamers.com.tw%2Fpuku-prod-zh%2Fanonymous-story%2F0167111e-9670-408f-b966-dfc727fda532.jpg',
                  previewImageUrl='https://thumbor.4gamers.com.tw/ScSRflOzjgQwfoiNq4AhSDvegwY=/adaptive-fit-in/1200x1200/filters:no_upscale():extract_cover():format(jpeg):quality(85)/https%3A%2F%2Fugc-media.4gamers.com.tw%2Fpuku-prod-zh%2Fanonymous-story%2F0167111e-9670-408f-b966-dfc727fda532.jpg')

    elif action =='video':
      reply = VideoMessage(originalContentUrl='https://youtu.be/dQw4w9WgXcQ',
                  previewImageUrl='https://i.ytimg.com/vi/dQw4w9WgXcQ/hq720.jpg?sqp=-oaymwEhCK4FEIIDSFryq4qpAxMIARUAAAAAGAElAADIQj0AgKJD&rs=AOn4CLB14ZAZXvYR0JMDHzRz7hdvw76w9Q'
        )

    elif action =='location':
        reply=LocationMessage(
        title="中山大學",
        address="804高雄市鼓山區蓮海路70號",
        latitude=22.624722,
        longitude=120.266111
        )
    else:
      response = ask_gemini(action)
      reply = TextMessage(text=response)

    line_bot_api.reply_message(
     ReplyMessageRequest(
        reply_token=event.reply_token,
        messages=[reply]
      )
    )

ngrok_start()
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://c2b9-35-196-74-180.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:48:04] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:48:12] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:48:21] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jun/2025 10:49:18] "POST /callback HTTP/1.1" 200 -
